In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np

# Load the MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()

# Normalize the dataset (pixel values between 0 and 1)
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Add a channel dimension to the dataset (required by Conv2D layers)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Check the shape
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")


In [ ]:
# Define the encoder
def build_encoder():
    encoder = models.Sequential()
    encoder.add(layers.Input(shape=(28, 28, 1)))
    encoder.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    encoder.add(layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
    encoder.add(layers.MaxPooling2D((2, 2), padding='same'))
    return encoder

# Define the decoder
def build_decoder():
    decoder = models.Sequential()
    decoder.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(7, 7, 16)))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
    return decoder

In [ ]:
# Build the autoencoder model by combining encoder and decoder

# Input shape for the autoencoder
input_img = layers.Input(shape=(28, 28, 1))

# Pass input through the encoder
encoded_img = encoder(input_img)

# Pass encoded image through the decoder
decoded_img = decoder(encoded_img)

# Define the autoencoder model using the functional API
autoencoder = models.Model(input_img, decoded_img)

# Compile the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Print the model summary
autoencoder.summary()


In [ ]:
# Train the autoencoder
history = autoencoder.fit(x_train, x_train, epochs=20, batch_size=256, validation_data=(x_test, x_test))

In [ ]:
# Plot the loss during training
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Compress and reconstruct some test images
encoded_imgs = encoder.predict(x_test)
decoded_imgs = autoencoder.predict(x_test)

# Display original, encoded (compressed), and reconstructed images
n = 10  # Number of images to display
plt.figure(figsize=(20, 6))
for i in range(n):
    # Display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title("Original")
    plt.axis('off')
    
    # Display encoded (compressed)
    ax = plt.subplot(3, n, i + n + 1)
    plt.imshow(encoded_imgs[i].reshape(7, 7 * 16), cmap='gray')  # Flattening the encoded image for visualization
    plt.title("Encoded")
    plt.axis('off')

    # Display reconstructed
    ax = plt.subplot(3, n, i + 2 * n + 1)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.title("Reconstructed")
    plt.axis('off')
plt.show()

In [ ]:
# Save the model for future use
autoencoder.save('autoencoder_image_compression.h5')
print("Model saved successfully!")